## Human Reference Genoma 

In [1]:
import pandas as pd
import pickle
from Bio import SeqIO

Load dataset

In [2]:
def load_data_from_pickle(path):
    # Load the pickle file
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

In [3]:
FILE_PATH = 'human_genoma/human_genoma_rna.pkl'
PROTEIN_FILE_PATH = 'human_genoma/human_genoma_protein.pkl'

human_genoma_df = load_data_from_pickle(FILE_PATH)
protein_df = load_data_from_pickle(PROTEIN_FILE_PATH)

In [4]:
human_genoma_df.head(2)

,ID,sequence
0,NR_168385.1,AGCAGGGCGTCCAGCGGAGAAGGCAGAGGAGGGGAGATGCGGGCTC...
1,NR_168384.1,AGCAGGGCGTCCAGCGGAGAAGGCAGAGGAGGGGAGATGCGGGCTC...


Nucleotides allocation of symbols:
- Guanine. adenine, thymine, cytosine: G,A,T,C
- Purine (adenine or guanine): R
- Pyrimidine (thymine or cytosine): Y
- Adenine or thymine: W
- Guanine or cytosine: S
- Adenine or cytosine: M
- Guanine or thymine: K
- Adenine or thymine or cytosine: H
- Guanine or cytosine or thymine: B
- Guanine or adenine or cytosine: V
- Guanine or adenine or thymine: D
- Guanine or adenine or thymine or cytosine: N

Explore sequences category:

In [5]:
human_genoma_df['category'] = human_genoma_df['ID'].str.split('_', expand=True)[0]
human_genoma_df.head(2)

,ID,sequence,category
0,NR_168385.1,AGCAGGGCGTCCAGCGGAGAAGGCAGAGGAGGGGAGATGCGGGCTC...,NR
1,NR_168384.1,AGCAGGGCGTCCAGCGGAGAAGGCAGAGGAGGGGAGATGCGGGCTC...,NR


In [6]:
human_genoma_df['category'].value_counts()

XM    131642
NM     53668
XR     51976
NR     23224
Name: category, dtype: int64

In [7]:
protein_df['category'] = protein_df['ID'].str.split('_', expand=True)[0]
protein_df['category'].value_counts()

XP    131642
NP     53668
YP        13
Name: category, dtype: int64

| Category | Description                                    | Comment                                          |
|----------|------------------------------------------------|--------------------------------------------------|
| NC       | Complete genomic molecules                     |                                                  |
| NG       | Incomplete genomic region                      |                                                  |
| NM       | mRNA                                           | Protein-coding transcripts (usually curated)      |
| NR       | ncRNA                                          | Non-protein-coding transcripts                    |
| NP       | Protein                                        | Associated with an NM_ or NC_ accession           |
| XM       | Predicted mRNA model                           | Predicted model protein-coding transcript         |
| XR       | Predicted ncRNA model                          | Predicted model non-protein-coding transcript     |
| YP       | Protein	                                    | Annotated on genomic molecules without an instantiated transcript record|
| XP       | Predicted Protein model (eukaryotic sequences) | Associated with an XM_ accession                  |
| WP       | Predicted Protein model (prokaryotic sequences)| Non-redundant across multiple strains and species |

In [8]:
categories = list(set(human_genoma_df['category'].unique()))+list(set(protein_df['category'].unique()))
categories

['XM', 'XR', 'NM', 'NR', 'NP', 'XP', 'YP']

In [11]:
columns = ['ID']
for category in categories:
    columns.append(category+'_ID')
    columns.append(category+'_sequence')

final_df = pd.DataFrame(columns=columns)

In [18]:
human_genoma_df.shape

(260510, 3)

In [20]:
def update_final_df(row):
    ID = row['ID'].split('_')[1]
    category = row['category']
    sequence = row['sequence']
    
    if ID in final_df['ID'].values:
        final_df.loc[final_df['ID'] == ID, category+'_ID'] = category+ID
        final_df.loc[final_df['ID'] == ID, category+'_sequence'] = sequence
    else:
        new_row = pd.DataFrame({'ID': ID, category+'_ID': category+ID, category+'_sequence': sequence}, index=[0])
        return pd.concat([final_df, new_row], ignore_index=True)

In [21]:
final_df = human_genoma_df.apply(update_final_df, axis=1).dropna().reset_index(drop=True)

KeyboardInterrupt: 

In [ ]:
final_df = protein_df.apply(update_final_df, axis=1).dropna().reset_index(drop=True)

In [ ]:
final_df